In [2]:
import numpy as np

def trinomial_call(S, X, r, sigma, T, N):
    dt = T / N
    u = np.exp(sigma * np.sqrt(2 * dt))
    d = 1 / u
    p = (np.exp(r * dt / 2) - np.exp(-sigma * np.sqrt(dt / 2))) / (np.exp(sigma * np.sqrt(dt / 2)) - np.exp(-sigma * np.sqrt(dt / 2)))
    q = 1 - p

    # Initialize the final asset prices at maturity
    S_values = [S * u**j * d**(N-j) for j in range(N+1)]

    # Initialize the option values at maturity
    V_values = [max(S_value - X, 0) for S_value in S_values]

    # Calculate the option values at previous time steps
    for i in range(N-1, -1, -1):
        V_next = V_values.copy()
        for j in range(i+1):
            V_values[j] = np.exp(-r * dt) * (p * V_next[j+1] + q * V_next[j])

    return V_values[0]

def binomial_call(S, X, r, sigma, T, N):
    dt = T / N
    u = np.exp(sigma * np.sqrt(dt))
    d = 1 / u
    p = (np.exp(r * dt) - d) / (u - d)
    q = 1 - p

    # Initialize the final asset prices at maturity
    S_values = [S * u**j * d**(N-j) for j in range(N+1)]

    # Initialize the option values at maturity
    V_values = [max(S_value - X, 0) for S_value in S_values]

    # Calculate the option values at previous time steps
    for i in range(N-1, -1, -1):
        V_next = V_values.copy()
        for j in range(i+1):
            V_values[j] = np.exp(-r * dt) * (p * V_next[j+1] + q * V_next[j])

    return V_values[0]

# Input parameters
S = 100
X = 100
r = 0.05
sigma = 0.2
T = 1
N_trinomial = 100
N_binomial = 20

# Evaluate European call option using trinomial model
trinomial_price = trinomial_call(S, X, r, sigma, T, N_trinomial)

# Evaluate European call option using CRR binomial model
binomial_price = binomial_call(S, X, r, sigma, T, N_binomial)

# Output results
print("Trinomial model price:", trinomial_price)
print("CRR binomial model price with 20 steps:", binomial_price)


Trinomial model price: 14.840280089321729
CRR binomial model price with 20 steps: 10.351260189053285


In [3]:
import math

def boyle_probabilities(T, r, sigma, N, lambd):
    dt = T / N
    u = math.exp(lambd * sigma * math.sqrt(dt))
    d = 1 / u
    p_up = ((math.exp(r * dt) - d) / (u - d)) * (1 / lambd**2)
    p_down = ((u - math.exp(r * dt)) / (u - d)) * (1 / lambd**2)
    p_middle = 1 - (p_up + p_down)
    return p_up, p_down, p_middle

# Input parameters
T = 0.5
r = 0.05/250
sigma = 0.01
N = 5
lambd = 1.25

# Calculate probabilities using Boyle's model
p_up, p_down, p_middle = boyle_probabilities(T, r, sigma, N, lambd)

# Compare probabilities to 1/3
expected_probability = 1/3

# Output results
print("Probability of moving up:", p_up)
print("Probability of moving down:", p_down)
print("Probability of staying in the middle:", p_middle)
print("Expected probability (1/3):", expected_probability)


Probability of moving up: 0.32098664342804145
Probability of moving down: 0.31901335657195856
Probability of staying in the middle: 0.36
Expected probability (1/3): 0.3333333333333333


In [6]:
import numpy as np
from scipy.stats import norm

def levy_call_price(S, K, r, T, alpha, beta, delta, gamma):
    # S: Current stock price
    # K: Strike price
    # r: Risk-free interest rate
    # T: Time to expiration
    # alpha, beta, delta, gamma: Parameters of NIG process

    # Calculate the parameters of the NIG distribution
    lambda_ = np.sqrt(alpha**2 - beta**2)
    mu = r - delta - gamma * lambda_

    # Calculate the characteristic exponent of the NIG process
    def psi(u):
        return -delta * u - gamma * np.sqrt(alpha**2 - (beta + u * 1j)**2)

    # Define the risk-neutral characteristic function
    def phi(u):
        return np.exp(T * psi(u * 1j)) * np.exp(1j * u * np.log(S))

    # Calculate the call option price using the Fourier inversion formula
    def inversion_formula(w):
        integrand = np.exp(-1j * w * np.log(K)) * phi(w - 1j) / (w**2 + 1)
        return np.real(integrand)

    integral = np.trapz([inversion_formula(w) for w in np.linspace(0, 100, 1000)])
    call_price = S * np.exp(-r * T) / np.pi + integral / np.pi

    return call_price

# Example usage
S = 400  # Current stock price
K = 1510  # Strike price
r = 0.05  # Risk-free interest rate
T = 1  # Time to expiration
alpha = 2  # Levy process parameter
beta = 1  # Levy process parameter
delta = 0  # Levy process parameter
gamma = 1  # Levy process parameter

option_price = levy_call_price(S, K, r, T, alpha, beta, delta, gamma)
print("Option Price:", option_price)


Option Price: 193.5513921472742
